This notebook is used to convert a model to the JSON format

#Setup

##Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import sys
sys.path.append('/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch')

import os
from image import *
from model import CSRNet
import glob
import torch

Mounted at /content/gdrive


##Variables
Paramaters to change according to the wanted output

In [ ]:
#parameters to change
INPUT_MODEL_NAME = 'A10S_SH_C.tar' #Name of the saved CSRNet model weight
OUTPUT_MODEL_NAME = 'A10S_SH_340x257' # Will be directory with the json and the bins, usable with tensorflowJS
Input_img_size = (340,257) #Size of the model predictions
use_gpu = False

#paths

In [ ]:
root = '/content/gdrive/MyDrive/TFE_crowd_counting'

scripts_path = os.path.join(root,'CSRNet-pytorch')
models_path = os.path.join(scripts_path,'models')
onnx_path = os.path.join(scripts_path,'models/ONNX')
input_model_path = os.path.join(models_path,INPUT_MODEL_NAME)
onnx_model_path = os.path.join(onnx_path, OUTPUT_MODEL_NAME+'.onnx')
tf_model_path = os.path.join(models_path, OUTPUT_MODEL_NAME+'.pb')
output_path = os.path.join(models_path, OUTPUT_MODEL_NAME)

#Utils

In [ ]:
#return the loaded model located at model_path (or the basic shangai partAmodel if no path is given ) 
#using gpu or using cpu (if the use_gpu parameter is set to False)
def load_model(model_path, use_gpu = use_gpu):
  from torchvision import datasets, transforms
  transform=transforms.Compose([
                        transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
                    ])
  model = CSRNet()
  checkpoint = torch.load(model_path)
  model.load_state_dict(checkpoint['state_dict'])
  if use_gpu:
    model = model.cuda()
  else:
    model = model.cpu()
  return model

#Conversion

In [ ]:
#Step 1: convert to onnx
use_gpu = False
model = load_model(input_model_path)
input_names = [ "image" ]
output_names = [ "output1" ]
if use_gpu:
  dummy_input = torch.randn(1, 3, Input_img_size[0], Input_img_size[1], device='cuda')
else:
  dummy_input = torch.randn(1, 3, Input_img_size[1], Input_img_size[0])
torch.onnx.export(model, dummy_input, onnx_model_path, operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:286: UserWarning: `add_node_names' can be set to True only when 'operator_export_type' is `ONNX`. Since 'operator_export_type' is not set to 'ONNX', `add_node_names` argument will be ignored.
  "`{}` argument will be ignored.".format(arg_name, arg_name))
/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:286: UserWarning: `do_constant_folding' can be set to True only when 'operator_export_type' is `ONNX`. Since 'operator_export_type' is not set to 'ONNX', `do_constant_folding` argument will be ignored.
  "`{}` argument will be ignored.".format(arg_name, arg_name))


In [ ]:
#Step 2: export to tensorflow
!pip install git+https://github.com/onnx/onnx-tensorflow.git #May be commented if the package is already installed
!onnx-tf convert -i {onnx_model_path} -o {tf_model_path}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/onnx/onnx-tensorflow.git to /tmp/pip-req-build-0phrm6zo
  Running command git clone -q https://github.com/onnx/onnx-tensorflow.git /tmp/pip-req-build-0phrm6zo
     |████████████████████████████████| 12.8 MB 9.7 MB/s 
     |████████████████████████████████| 1.1 MB 55.7 MB/s 
  Created wheel for onnx-tf: filename=onnx_tf-1.10.0-py3-none-any.whl size=226602 sha256=44f71ca34c9c16c31561df3ae74f1e1a714bd7b1270e82b3fa407f7508790a57
  Stored in directory: /tmp/pip-ephem-wheel-cache-v93028c2/wheels/a4/ff/a6/e35bd7f47530fc3b199ee3e5641b0a144fbc09292fc9420e5c
Successfully built onnx-tf
2022-05-24 09:31:50,627 - onnx-tf - INFO - Start converting onnx pb to tf saved model
2022-05-24 09:31:54.255223: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config

In [ ]:
#Step 3: export to tensorflow.js
!pip install tensorflowjs #May be commented if the package is already installed

  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
#Copy past the two path as argument for conversion
print(tf_model_path)
print(output_path)

/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/models/A10S_SH_340x257.pb
/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/models/A10S_SH_340x257


In [ ]:
!tensorflowjs_converter --input_format='tf_saved_model' '/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/models/A10S_SH_340x257.pb' '/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/models/A10S_SH_340x257'

2022-05-24 09:32:13.728244: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
